In [1]:
import pandas as pd
import nibabel as nb
import os
import numpy as np
import glob
import h5py
import hcp_utils as hcp
from sklearn.random_projection import SparseRandomProjection

import numpy as np
from stacking_fmri import stacking_CV_fmri, stacking_fmri
from ridge_tools import R2
import matplotlib.pyplot as plt
import seaborn as sns

import time
#pd.set_option('display.max_rows', None)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
def load_audio_features(stim,delay,all_layers):
    #dimensionality reduction to 50 components
    transformer = SparseRandomProjection(n_components=50)
    

    save_features_dir = f'../data/{stim}_clips_cochresnet50/'
    
    X=[]
    # X_train=[]
    # X_test=[]
    #print('CochResNet50 time-averaged')
    # Open the file 'myfile.h5' in read-only mode
    file = h5py.File(f'{save_features_dir}cochresnet50_activations.h5', 'r')
    for layer in all_layers:
    # # Now you can access datasets within the file
        data = file[layer]
        #print(data.shape, layer)
        X.append(  transformer.fit_transform(  np.array(data)[:(-1*delay),:]  )  )
        # X_train.append(np.array(data)[:600,:])
        # X_test.append(np.array(data)[600:,:])
    
    # # Don't forget to close the file when you're done
    file.close()
    return(X)

def load_video_features(stim,delay,all_layers):
    #dimensionality reduction to 50 components
    transformer = SparseRandomProjection(n_components=50)
    save_path = f'../data/{stim}_frames_resnet50/'
    #print('ResNet50 frame embeddings')
    # for emb_f in emb_list:
    #     emb = np.load(f'{emb_f}')
    #     print(emb['frame_0000.jpg'].shape, os.path.splitext(os.path.basename(emb_f))[0])
    X=[]
    for layer in all_layers:
        X_layer=[]
        emb = np.load(f'{save_path}{layer}.npz')
        for k in list(emb.keys()):
            X_layer.append(emb[k].flatten())
        X.append(  transformer.fit_transform(  np.array(X_layer)[:(-1*delay),:]  )  )
    return(X)

def get_parcel_indices(parcels):
    patternR = '|'.join(['Right_' + parcel for parcel in parcels])
    patternL = '|'.join(['Left_' + parcel for parcel in parcels])
    
    # get a boolean series where True indicates a match
    matches = atlas['label'].str.contains(patternR) | atlas['label'].str.contains(patternL)
    
    # get the indices that match
    indices = matches[matches].index.tolist()
    parcel_names=atlas[matches]['label'].tolist()
    return(indices,parcel_names)


def load_fmri_data(im_file,delay,indices):
    # load fmri data for a subject
    img = nb.load(im_file)
    img_y = img.get_fdata()
    Y=img_y[delay:,indices]
    return(Y)
    # Y_train=Y[:600,:]
    # Y_test=Y[600:,:]

def get_subject_list(delay,indices):
    pilot_subjects=pd.read_csv('../data/pilots_ru_dm.csv') # load pilot subjects
    
    subjects=[]
    for sub in list(pilot_subjects['Identifiers_y']):
        
        im_file = f'/nese/mit/group/sig/projects/hbn/hbn_bids/derivatives/xcp_d_0.7.1/sub-{sub}/ses-HBNsiteRU/func/sub-{sub}_ses-HBNsiteRU_task-movieDM_space-fsLR_seg-Glasser_den-91k_stat-mean_timeseries.ptseries.nii'
        try:
            Y=load_fmri_data(im_file,delay,indices)
            #print('loaded')
            subjects.append(sub)
        except:
            print(f'missing {sub}')
    print(f'loaded {len(subjects)} subjects')
    return(subjects)
    
def get_parcel_indices(parcels):
    patternR = '|'.join(['Right_' + parcel for parcel in parcels])
    patternL = '|'.join(['Left_' + parcel for parcel in parcels])
    
    # get a boolean series where True indicates a match
    matches = atlas['label'].str.contains(patternR) | atlas['label'].str.contains(patternL)
    
    # get the indices that match
    atlas_indices = atlas[matches]['index'].tolist()
    indices = matches[matches].index.tolist()
    parcel_names=atlas[matches]['label'].tolist()
    return(atlas_indices,indices,parcel_names)

In [3]:
def load_glasser():
    atlas_dlabel='/om2/user/jsmentch/atlases/atlas-Glasser/atlas-Glasser_space-fsLR_den-91k_dseg.dlabel.nii'
    img = nb.load(atlas_dlabel)
    atlas_data=img.get_fdata()
    atlas_data=atlas_data[0,:]
    atlas=pd.read_csv('/om2/user/jsmentch/atlases/atlas-Glasser/atlas-Glasser_dseg.tsv', sep='\t')
    return(atlas,atlas_data)
    
atlas,atlas_data=load_glasser()
parcels=[
    'A1']
atlas_indices,indices,parcel_names=get_parcel_indices(parcels)

In [4]:
atlas_indices

[24, 204]

In [5]:
parcel_names

['Right_A1', 'Left_A1']

In [6]:
atlas_indices_indices=np.where(atlas_data==204)[0]

In [7]:
atlas_indices_indices

array([ 6776,  6777,  6778,  6779,  6780,  8297,  8303,  8304,  8310,
       28416, 28417, 28418, 28419, 28465, 28466, 28467, 28468, 28469,
       28470, 28512, 28513, 28514, 28515, 28516, 28517, 28518, 28519,
       28559, 28560, 28561, 28562, 28563, 28564, 28565, 28566, 28567,
       28605, 28606, 28607, 28608, 28609, 28610, 28611, 28612, 28613,
       28614, 28650, 28651, 28652, 28653, 28654, 28655, 28656, 28657,
       28658, 28659, 28660, 28695, 28696, 28697, 28698, 28699, 28700,
       28701, 28702, 28703, 28704, 28705, 28740, 28741, 28742, 28743,
       28744, 28745, 28746, 28747, 28748])

In [8]:
delay=6
subjects=get_subject_list(delay,indices)

missing NDARZN277NR6
missing NDARJG300HWH
loaded 25 subjects


In [9]:
sub=subjects[0]
im_file = f'/nese/mit/group/sig/projects/hbn/hbn_bids/derivatives/xcp_d_0.7.1/sub-{sub}/ses-HBNsiteRU/func/sub-{sub}_ses-HBNsiteRU_task-movieDM_space-fsLR_den-91k_desc-denoisedSmoothed_bold.dtseries.nii'
img = nb.load(im_file)
img_y = img.get_fdata()
Y=img_y[delay:,atlas_indices_indices]

In [10]:
def load_audio_features_RAW(stim,delay,all_layers):
    #dimensionality reduction to 50 components
    #transformer = SparseRandomProjection(n_components=50)
    

    save_features_dir = f'../data/{stim}_clips_cochresnet50/'
    
    X=[]
    # X_train=[]
    # X_test=[]
    #print('CochResNet50 time-averaged')
    # Open the file 'myfile.h5' in read-only mode
    file = h5py.File(f'{save_features_dir}cochresnet50_activations.h5', 'r')
    for layer in all_layers:
    # # Now you can access datasets within the file
        data = file[layer]
        print(data.shape, layer)
        #X.append(  transformer.fit_transform(  np.array(data)[:(-1*delay),:]  )  )
        X.append( np.array(data)[:(-1*delay),:]  )

        # X_train.append(np.array(data)[:600,:])
        # X_test.append(np.array(data)[600:,:])
    
    # # Don't forget to close the file when you're done
    file.close()
    return(X)

In [31]:
all_layers=['input_after_preproc',
 'conv1_relu1',
 'maxpool1',
 'layer1',
 'layer2',
 'layer3',
 'layer4',
 'avgpool']

# all_layers=['input_after_preproc',
#  'conv1_relu1',
#  'maxpool1']
#X=load_audio_features_RAW('DM',delay,all_layers)
X=load_audio_features('DM',delay,all_layers)

X = [array[:Y.shape[0], :] for array in X]
Y= Y[:X[0].shape[0],:]

In [32]:
#X[0].shape
#Y.shape

In [33]:
import numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

# Prepare your data (if not already done)
# X = np.array(X)
# Y = np.array(Y)

for lay in np.arange(len(X)):
# Create a RidgeCV model
#alphas = np.logspace(-6, 6, 13)
    ridge_cv = RidgeCV( cv=5)
    
    # Fit the model to your data
    ridge_cv.fit(X[lay], Y)
    
    # Predict Y using cross-validation
    Y_pred = cross_val_predict(ridge_cv, X[lay], Y, cv=5)
    
    # Evaluate the model
    mse = mean_squared_error(Y, Y_pred)
    r2 = r2_score(Y, Y_pred)
    print(f"MSE: {mse:.2f}, R2: {r2:.2f}")


MSE: 1.79, R2: -0.00
MSE: 2.41, R2: -0.34
MSE: 2.36, R2: -0.32
MSE: 2.44, R2: -0.38
MSE: 2.36, R2: -0.33
MSE: 2.38, R2: -0.33
MSE: 2.26, R2: -0.27
MSE: 2.24, R2: -0.26


In [ ]:
r2s, stacked_r2s, _, _, _, S_average = stacking_CV_fmri(Y, X, method = 'cross_val_ridge',n_folds = 5,score_f=R2)

In [ ]:
S_average.shape

In [ ]:
#plt.plot( S_average, linewidth=0.5 )
fig = plt.figure(figsize=(10, 2))

np.arange(S_average.shape[1])

sns.heatmap(S_average.T, vmin=0, vmax=1)
plt.xlabel('Voxel ID')
plt.ylabel('Feature')
plt.yticks(np.arange(S_average.shape[1])+0.5,all_layers, rotation=0)

In [ ]:
fig, axs = plt.subplots(r2s.shape[0], 1, figsize=(5, 1*r2s.shape[0]))  # Adjust the figure size as needed

for layer in np.arange(r2s.shape[0]):
    data = r2s[layer]
    axs[layer].hist(data, 150, color='k')
    axs[layer].set_xlim(r2s.min(), r2s.max())
    axs[layer].set_xlabel('r2')
    axs[layer].set_ylabel('grayordinates')

plt.tight_layout()  # To prevent overlap of subplots
plt.show()

In [66]:
import nibabel as nib
import nilearn.plotting as plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import hcp_utils as hcp


In [67]:
img = nb.load(atlas_dlabel)
atlas_data=img.get_fdata()
atlas_data=atlas_data[0,:]

atlas_data = np.where(atlas_data < 6, 1, 0)
plotting.view_surf(hcp.mesh.inflated, hcp.cortex_data(atlas_data), bg_map=hcp.mesh.sulc)

NameError: name 'atlas_dlabel' is not defined